In [1]:
from ImageNode import ImageNode
import argparse
import logging
import os
from pathlib import Path
from scipy.io import loadmat
from PIL import Image
import numpy as np
from tqdm import tqdm
from itertools import chain

%load_ext autoreload
%autoreload 2

In [2]:
DISTANCE_THRESHOLD_PERCENTILE = 90
RANSAC_THRESHOLD = 5
SCALE_FACTOR = 0.5

In [3]:
ref_dir = Path("/home/davigv/ist/piv/piv-hw/Part1_2/delivery/Pplant/ref_dir")
input_output_pairs = [
    (Path("/home/davigv/ist/piv/piv-hw/Part1_2/delivery/Pplant/input1_dir"),
    Path("/home/davigv/ist/piv/piv-hw/Part1_2/delivery/Pplant/output1_dir")),
    (Path("/home/davigv/ist/piv/piv-hw/Part1_2/delivery/Pplant/input2_dir"),
    Path("/home/davigv/ist/piv/piv-hw/Part1_2/delivery/Pplant/output2_dir"))
]

In [4]:
from load_data import load_reference_node


ref_node = load_reference_node(Path(ref_dir))


In [5]:
from load_data import process_dir_pairs


image_nodes = process_dir_pairs(input_output_pairs)
for i, node in enumerate(image_nodes):
    node.image_num = i

In [ ]:
from thresholds import max_descriptor_distance_threshold_numpy


max_desc_dist = max_descriptor_distance_threshold_numpy(image_nodes, DISTANCE_THRESHOLD_PERCENTILE)
print('The maximum descriptor distance is:', round(max_desc_dist, 4))

In [ ]:
from thresholds import compute_min_inliers_threshold


min_inliers_threshold = compute_min_inliers_threshold(image_nodes, max_desc_dist)
print('The minimum number of inliers is:', min_inliers_threshold)

In [ ]:
from homography import find_direct_homographies


valid_homographies = find_direct_homographies(image_nodes, ref_node, min_inliers_threshold, max_desc_dist)
if not valid_homographies[1]:
    raise ValueError("No valid direct homographies found.")

In [ ]:
from homography import find_remaining_homographies


find_remaining_homographies(image_nodes, valid_homographies, min_inliers_threshold, max_desc_dist)

In [ ]:
print('Footprint of each node:')
for node in image_nodes:
    print(node.footprint)

In [ ]:
print("Transforming YOLO")
for node in image_nodes:
    node.transform_yolo_detections()

In [ ]:
# from matplotlib import pyplot as plt
# from plot import create_canvas_with_images

# # Example usage:
# final_canvas = create_canvas_with_images(ref_node.image, image_nodes)

# # Display the final canvas
# plt.figure(figsize=(20, 20))
# plt.imshow(final_canvas)
# plt.title("Final Canvas with All Images in Reference Frame")
# plt.axis('off')

# # Save the canvas to the current directory
# plt.savefig('final_canvas.png', bbox_inches='tight')
# plt.show()